## SSL Method: SimCLR (Multimodal)

In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../../") # Add the parent directory to the path
from utils.reproducibility import set_seed
set_seed(1)
import torch
import os
from utils.get_data import AVMNISTDataModule, AVMNISTSimCLRDataModule
import torch.nn as nn
import torch.optim as optim
from models.dino import SpectrogramEncoder
import torch.nn.functional as F
import lightning.pytorch as pl
from models.dino import ImageEncoder, SpectrogramEncoder, \
                        ProjectionHead
from training_structures.ssl_train import train_and_evaluate_ssl
from multimodal_simclr import MultiModalSimCLRModel, MultiModalSimCLRLightning

current_path = os.getcwd()
root_path = "../../"
data_dir=f'{root_path}/data/avmnist/'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
train_data_module = AVMNISTSimCLRDataModule(data_dir=data_dir, batch_size=256, num_workers=6)
test_data_module = AVMNISTDataModule(data_dir=data_dir, batch_size=128, num_workers=0)
num_epochs = 100
model_dir, model_name = f"{current_path}", "multimodal_simclr_model"
module = MultiModalSimCLRLightning(projection_dim=256, output_dim=256, learning_rate=0.001, num_epochs=num_epochs, use_mixed_precision=True)
seeds = [1, 2, 3]
input_data = [ # img1, spec1, img2, spec2
    (torch.randn(1, 1, 28, 28),
    torch.randn(1, 1, 112, 112),  
    torch.randn(1, 1, 28, 28),
    torch.randn(1, 1, 112, 112))      
]

train_and_evaluate_ssl(
    module=module,
    train_data_module=train_data_module,
    test_data_module=test_data_module,
    model_dir=model_dir,
    model_name=model_name,
    input_data=input_data,
    modalities=["audio", "image"],
    seeds=[1, 2, 3],
    num_epochs=num_epochs
)

c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\lightning\fabric\connector.py:572: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\lightning\pytorch\trainer\configuration_validator.py:68: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                  | Params | Mode 
--------------------------------------------------------
0 | model | MultiModalSimCLRModel | 1.3 M  | train
--------------------------------------------------------
1.3 M     Trainable params
0         Non-trainable params
1.3 M     Total params
5.092     Total estimated model params size (MB)
55        Modules in train mode


Model GFLOPs: 0.19 GFLOPs
Model Parameters: 1.27 M parameters
Training with seed 1...
Epoch 99: 100%|██████████| 215/215 [01:09<00:00,  3.08it/s, v_num=eed1, train_loss_step=0.255, train_loss_epoch=2.650]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 215/215 [01:09<00:00,  3.08it/s, v_num=eed1, train_loss_step=0.255, train_loss_epoch=2.650]
Extracting training features...


100%|██████████| 430/430 [00:17<00:00, 24.33it/s]


...

Visualizing digits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


c:\Users\Ward\Desktop\vub-github\Thesis-project\AVMNIST_Experiments\other_ssl\multimodal_simclr\../..\utils\visualisations.py:750: UserWarning: Tight layout not applied. The left and right margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


100%|██████████| 79/79 [00:01<00:00, 50.48it/s]


Visualizing digits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Applying t-SNE...


c:\Users\Ward\anaconda3\envs\multibench\Lib\site-packages\sklearn\manifold\_t_sne.py:1162: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
